In [1]:
import pandas as pd 


dfs = pd.read_html('https://en.wikipedia.org/wiki/Russell_1000_Index')
print(f'there are {len(dfs)} tables')

there are 6 tables


In [2]:
df = None

for d in dfs:
    if 'Symbol' in set(d.columns.to_list()):
        df = d 

assert df is not None

df.head()

,Company,Symbol,GICS Sector,GICS Sub-Industry
0,10x Genomics,TXG,Health Care,Health Care Technology
1,3M,MMM,Industrials,Industrial Conglomerates
2,A. O. Smith,AOS,Industrials,Building Products
3,AAON,AAON,Industrials,Building Products
4,Abbott Laboratories,ABT,Health Care,Health Care Equipment


In [3]:
df.tail()

,Company,Symbol,GICS Sector,GICS Sub-Industry
997,Zions Bancorporation,ZION,Financials,Regional Banks
998,Zoetis,ZTS,Health Care,Pharmaceuticals
999,Zoom Communications,ZM,Information Technology,Application Software
1000,ZoomInfo,ZI,Communication Services,NaN
1001,Zscaler,ZS,Information Technology,Application Software


In [4]:
print(f'there are {df.shape[0]} companies in Russell 1000 index')

there are 1002 companies in Russell 1000 index


In [5]:
from sqlalchemy.orm import Session

from src.utils import get_db_engine
from src.create_tables import RUSSELL1000

table = RUSSELL1000

engine = get_db_engine()
session = Session(engine)

count = session.query(table).count()
print(f'there are {count} rows in table {table.__tablename__}.')

there are 1002 rows in table russell_1000.


In [6]:
for column in table.__table__.columns:
    print(f'Column: {column.name}, Type: {column.type}, Nullable: {column.nullable}')

Column: id, Type: INTEGER, Nullable: False
Column: ticker, Type: VARCHAR(10), Nullable: False
Column: company_name, Type: VARCHAR(100), Nullable: False
Column: sector, Type: VARCHAR(100), Nullable: False
Column: sub_industry, Type: VARCHAR(100), Nullable: False


In [7]:
df.rename(columns={'Company': 'company_name',
                   'Symbol': 'ticker',
                   'GICS Sector': 'sector',
                   'GICS Sub-Industry': 'sub_industry'}, inplace=True)

df.head()

,company_name,ticker,sector,sub_industry
0,10x Genomics,TXG,Health Care,Health Care Technology
1,3M,MMM,Industrials,Industrial Conglomerates
2,A. O. Smith,AOS,Industrials,Building Products
3,AAON,AAON,Industrials,Building Products
4,Abbott Laboratories,ABT,Health Care,Health Care Equipment


In [8]:
for c in df.columns: # type: ignore
    print(f'column {c} contains null value: {df[c].isna().any()}')

column company_name contains null value: False
column ticker contains null value: False
column sector contains null value: False
column sub_industry contains null value: True


In [9]:
df[df['sub_industry'].isna()]

,company_name,ticker,sector,sub_industry
10,ADT Inc.,ADT,Consumer Discretionary,NaN
12,Advanced Drainage Systems,WMS,Industrials,NaN
17,Affirm Holdings,AFRM,Information Technology,NaN
21,Agree Realty,ADC,Real Estate,NaN
22,AGNC Investment,AGNC,Financials,NaN
...,...,...,...,...
979,WillScot Mobile Mini,WSC,Industrials,NaN
980,Wingstop,WING,Consumer Discretionary,NaN
994,Zillow (Class A),ZG,Real Estate,NaN
995,Zillow (Class C),Z,Real Estate,NaN


In [10]:
df = df.fillna(value='unknown')

for c in df.columns:
    assert df[c].notna().all()

In [11]:
# from sqlalchemy import String

# assert df is not None

# df.to_sql('russell_1000', con=engine, index=False, if_exists='append',
#           dtype={'ticker': String(10), 
#                  'company_name': String(100), 
#                  'sector': String(100), 
#                  'sub_industry': String(100)}) # type: ignore

In [12]:
df.to_csv('index/russell_1000.csv', index=False)